In [ ]:
import os, pandas as pd, networkx as nx, numpy as np, statsmodels.api as sm
from sqlalchemy import create_engine
from datetime import date, datetime, timedelta
import matplotlib.pyplot as plt, seaborn as sns
plt.rcParams["mathtext.fontset"] = "custom"
plt.rcParams["mathtext.rm"] = "Arial"  # Use Arial for mathtext

## parameters

In [ ]:
yeari, yearf = '2024', '2024'
weeki, weekf = '18', '31'

In [ ]:
di = datetime.strptime(f'{yeari}-{weeki}-1', "%Y-%W-%w").date()
df = datetime.strptime(f'{yearf}-{weekf}-1', "%Y-%W-%w").date() + timedelta(6)
ds = [di+timedelta(dt) for dt in range((df-di).days+1)]
daylist = ds
print(di, 'until', df)

In [ ]:
cdef = 'tl7_10m'# 'tl5_10m' 'tl6_10m' 'tl7_10m' 'tl8_10m' 'tl8_60m'
cdef_alt = '16m_10min'# tl5: 62 ... tl7: 16   tl8: 8

## load data

In [ ]:
# n_c cities: load total contact numbers numbers for cities
data = pd.read_csv(f'output/00_ncontacts_cities_{cdef}.csv')
data['day'] = [d.date() for d in pd.to_datetime(data.day)]

# n_pop cities & stadiums: load stadium capacity data for EURO 2024
stadium_data = pd.read_csv('output/00_stadium_data.csv')

# n_d cities: load did numbers for cities
panel_data = pd.read_csv('output/00_panel_data.csv')
panel_data = panel_data.merge(stadium_data[['city','population']])
panel_data['pdid'] = panel_data.ndids / panel_data.population

# n_p cities: load ping numbers per did for cities
data_pingfreq = pd.read_csv('output/00_data_pingfreq.csv')
data_pingfreq['fpingsperdid'] = data_pingfreq.npingsperdid / 144.

# n_d stadiums: load did numbers for stadiums
panelstad_data2 = pd.read_csv('output/00_panelstad_data2.csv')
panelstad_data2['day'] = [d.date() for d in pd.to_datetime(panelstad_data2.day)]

# load mass event data
match_data = pd.read_csv('output/00_event_data.csv')
match_data['day'] = [d.date() for d in pd.to_datetime(match_data.day)]

## scaling between $n_c$ and $n_d$ ...

### ... for cities

In [ ]:
pdid_avg = panel_data[['city','pdid']].drop_duplicates().pdid.mean()
pdid_avg

In [ ]:
pping_avg = data_pingfreq[['city','fpingsperdid']].drop_duplicates().fpingsperdid.mean()
pping_avg

In [ ]:
#conpop_cmp = data.merge(stadium_data, on=['area_id','city']).merge(panel_data[['city','pdid']], on='city')
conpop_cmp = data.drop(columns=['area_id']).groupby(['day','city']).sum().reset_index()\
                .merge(panel_data[['city','pdid','ndids','population']], on='city')\
                .merge(data_pingfreq[['city','fpingsperdid']], on='city')\
                .drop_duplicates()
#conpop_cmp['ncontacts_1_normed'] = conpop_cmp.ncontacts_1 / conpop_cmp.pdid**1.2628 / (conpop_cmp.population / 1e5)**1.1125#**1.0387 **1.0743
#conpop_cmp['ncontacts_1_normed'] = conpop_cmp.ncontacts_1 / conpop_cmp.pdid / (conpop_cmp.population / 1e5)
#conpop_cmp['ncontacts_1_normed'] = conpop_cmp.ncontacts_1 / conpop_cmp.pdid / conpop_cmp.ndids# originally used in event cmp
#conpop_cmp['ncontacts_1_normed'] = conpop_cmp.ncontacts_1 / pdid_avg / conpop_cmp.ndids

#print('expo_city', expo_city)
#conpop_cmp['ncontacts_1_normed'] = conpop_cmp.ncontacts_1 / conpop_cmp.ndids# / conpop_cmp.pdid**(expo_city-1.) / pdid_avg

conpop_cmp

In [ ]:
sns.set_theme(style="ticks")

cmap = sns.color_palette('flare', as_cmap=True)#sns.cubehelix_palette(rot=-.2, as_cmap=True)
g = sns.relplot(
    data=conpop_cmp,
    x="population", y="ncontacts_1",
    #x="population", y="ncontacts_1_normed",
    #x="population", y="ncontacts_1_model",
    hue="pdid", size="fpingsperdid",#"ncontacts_2",
    palette=cmap, sizes=(10, 50),#200),
)
g.set(xscale="log", yscale="log")
g.ax.xaxis.grid(True, "minor", linewidth=.25)
g.ax.yaxis.grid(True, "minor", linewidth=.25)
g.despine(left=True, bottom=True)
g.ax.set_xlabel('city population')
g.ax.set_ylabel('detected contacts (outside stadiums)')
g.ax.grid(which='major', linestyle='-', linewidth='0.5', color='black', alpha=.25)
g.ax.grid(which='minor', linestyle='-', linewidth='0.5', color='grey', alpha=.25)
g.ax.set_xlim([2.01e5,4e6])
g.ax.set_ylim([1e2,3.99e3])# for tl7
#g.ax.set_ylim([4e2,1.99e4])# for tl5
#g.ax.set_ylim([4e1,1.99e3])# for tl8 10min
#g.ax.set_ylim([2e2,9.99e3])# for tl8 60min
g.ax.set_xticks([3e5,1e6,3e6])
g.ax.set_xticklabels([r'$3\times 10^5$', r'$10^6$', r'$3\times 10^6$'], fontname="Arial")

lg = g._legend
for tx in lg.texts:
    if tx.get_text() == 'pdid':
        tx.set_text('user share')
    if tx.get_text() == 'ncontacts_2':
        tx.set_text('in stadiums')
    if tx.get_text() == 'fpingsperdid':
        tx.set_text('ping rate')

plt.savefig(f'plots/fig2_{cdef_alt}/04_citybias_npop.jpg', bbox_inches='tight', dpi=300)
plt.savefig(f'plots/fig2_{cdef_alt}/04_citybias_npop.pdf', bbox_inches='tight')
plt.show()

In [ ]:
cmap = sns.color_palette('flare', as_cmap=True)#sns.cubehelix_palette(rot=-.2, as_cmap=True)
g = sns.relplot(
    data=conpop_cmp,
    x="pdid", y="ncontacts_1",#_normed",
    hue="population", size="fpingsperdid",#"ncontacts_2",
    palette=cmap, sizes=(10, 50),#200),
)
g.set(xscale="log", yscale="log")
g.ax.xaxis.grid(True, "minor", linewidth=.25)
g.ax.yaxis.grid(True, "minor", linewidth=.25)
g.despine(left=True, bottom=True)
g.ax.set_xlabel('user share')
g.ax.set_ylabel('detected contacts (outside stadiums)')
g.ax.grid(which='major', linestyle='-', linewidth='0.5', color='black', alpha=.25)
g.ax.grid(which='minor', linestyle='-', linewidth='0.5', color='grey', alpha=.25)
g.ax.set_xlim([8.01e-4,7e-3])
g.ax.set_ylim([1e2,3.99e3])# for tl7
#g.ax.set_ylim([4e2,1.99e4])# for tl5
#g.ax.set_ylim([4e1,1.99e3])# for tl8 10min
#g.ax.set_ylim([2e2,9.99e3])# for tl8 60min

lg = g._legend
for tx in lg.texts:
    if tx.get_text() == 'ncontacts_2':
        tx.set_text('in stadiums')
    if tx.get_text() == 'fpingsperdid':
        tx.set_text('ping rate')

plt.savefig(f'plots/fig2_{cdef_alt}/04_citybias_pdid.jpg', bbox_inches='tight', dpi=300)
plt.savefig(f'plots/fig2_{cdef_alt}/04_citybias_pdid.pdf', bbox_inches='tight')
plt.show()

In [ ]:
cmap = sns.color_palette('flare', as_cmap=True)#sns.cubehelix_palette(rot=-.2, as_cmap=True)
g = sns.relplot(
    data=conpop_cmp,
    x="fpingsperdid", y="ncontacts_1",#_normed",
    hue="pdid", size="population",#"ncontacts_2",
    palette=cmap, sizes=(10, 50),#200),
)
g.set(xscale="log", yscale="log")
g.ax.xaxis.grid(True, "minor", linewidth=.25)
g.ax.yaxis.grid(True, "minor", linewidth=.25)
g.despine(left=True, bottom=True)
g.ax.set_xlabel('ping frequency')
g.ax.set_ylabel('detected contacts (outside stadiums)')
g.ax.grid(which='major', linestyle='-', linewidth='0.5', color='black', alpha=.25)
g.ax.grid(which='minor', linestyle='-', linewidth='0.5', color='grey', alpha=.25)
#g.ax.set_xlim([8.01e-4,7e-3])
g.ax.set_ylim([1e2,3.99e3])# for tl7
#g.ax.set_ylim([4e2,1.99e4])# for tl5
#g.ax.set_ylim([4e1,1.99e3])# for tl8 10min
#g.ax.set_ylim([2e2,9.99e3])# for tl8 60min

lg = g._legend
for tx in lg.texts:
    if tx.get_text() == 'ncontacts_2':
        tx.set_text('in stadiums')
    if tx.get_text() == 'pdid':
        tx.set_text('user share')

plt.savefig(f'plots/fig2_{cdef_alt}/04_citybias_pping.jpg', bbox_inches='tight', dpi=300)
plt.savefig(f'plots/fig2_{cdef_alt}/04_citybias_pping.pdf', bbox_inches='tight')
plt.show()

In [ ]:
# Define the response variable y and the predictor variables X
y = conpop_cmp['ncontacts_1']
#y = conpop_cmp['ncontacts_2']

#X = conpop_cmp[['pdid', 'population']]
#X = conpop_cmp[['population']]
#X = conpop_cmp[['ndids','pdid']]
X = conpop_cmp[['population','pdid','fpingsperdid']]
X = conpop_cmp[['ndids','fpingsperdid']]
#X = conpop_cmp[['ndids_stad']]
#X['pdid'] = X.pdid / pdid_avg

y, X = np.log10(y), np.log10(X)
 
# Add a constant term to the predictors (for the intercept)
X = sm.add_constant(X)
 
# Fit the model
model = sm.OLS(y, X).fit()

# Print the model summary
print(model.summary())

In [ ]:
1/0.02

In [ ]:
1/(pping_avg**3.5)

In [ ]:
# Predict values using the model
#Xnew = X.copy(deep=True)
#Xnew['pdid'] = 0.

predictions = model.predict(X)
conpop_cmp['ncontacts_1_model'] = 10.**predictions
 
# Plot true values vs predicted values
plt.scatter(y, predictions)
plt.xlabel('True Values')
plt.ylabel('Predicted Values')
plt.title('True vs Predicted Values')
plt.plot([min(y), max(y)], [min(y), max(y)], color='red', linestyle='--')  # Line of perfect prediction
plt.show()

In [ ]:
cmap = sns.color_palette('flare', as_cmap=True)#sns.cubehelix_palette(rot=-.2, as_cmap=True)
g = sns.relplot(
    data=conpop_cmp,
    x="ncontacts_1_model", y="ncontacts_1",
    hue="pdid", size="fpingsperdid",
    palette=cmap, sizes=(10, 50),#200),
    lw=0,# size=10,
)
#g.ax.plot(conpop_cmp.ndids, 10.**(conpop_cmp.ncontacts_1_model), c='k', lw=3)
g.ax.plot([1e0,1e5], [1e0,1e5], c='k', lw=3)
g.set(xscale="log", yscale="log")
g.ax.xaxis.grid(True, "minor", linewidth=.25)
g.ax.yaxis.grid(True, "minor", linewidth=.25)
g.despine(left=True, bottom=True)
g.ax.set_xlabel('predicted contacts (outside stadiums)')#'users in city')
g.ax.set_ylabel('detected contacts (outside stadiums)')
g.ax.grid(which='major', linestyle='-', linewidth='0.5', color='black', alpha=.25)
g.ax.grid(which='minor', linestyle='-', linewidth='0.5', color='grey', alpha=.25)
#g.ax.set_xlim([1e3,7.99e3])
xu, xo = conpop_cmp.ncontacts_1_model.min() / 1.2, conpop_cmp.ncontacts_1_model.max() * 1.2
yu, yo = conpop_cmp.ncontacts_1.min() / 1.2, conpop_cmp.ncontacts_1.max() * 1.2
#g.ax.set_xlim([1./1.2*conpop_cmp[conpop_cmp.ncontacts_1>0.].ndids.min(),1.2*conpop_cmp.ndids.max()])
#g.ax.set_ylim([2.01e2,7.99e3])
g.ax.set_xlim([xu, xo])
g.ax.set_ylim([yu, yo])
g.ax.set_xticks([3e2,1e3,3e3])
g.ax.set_xticklabels([r'$3\times 10^2$', r'$10^3$', r'$3\times 10^3$'], fontname="Arial")
g.ax.set_yticks([3e2,1e3,3e3])
g.ax.set_yticklabels([r'$3\times 10^2$', r'$10^3$', r'$3\times 10^3$'], fontname="Arial")

##lg = g._legend
#lg.remove()
##for tx in lg.texts:
##    #if tx.get_text() == 'pdid':
##    #    tx.set_text('user share')
##    if tx.get_text() == '10':
##        tx.set_text('')
##lg.set_title('user share')

# Retrieve the legend handles and labels
##handles, labels = lg.legend_handles, lg.texts
#print([l.get_text() for l in labels])
# Define the label you want to remove
##label_to_remove = ''
# Filter out the handle and label to remove
##filtered_handles_labels = [(h, l) for h, l in zip(handles, labels) if l.get_text() != label_to_remove]
#print(filtered_handles_labels)
# Unpack the filtered handles and labels
##filtered_handles, filtered_labels = zip(*filtered_handles_labels)
# Update the legend with the filtered handles and labels
#g.ax.legend(filtered_handles, [label.get_text() for label in filtered_labels])

lg = g._legend
for tx in lg.texts:
    if tx.get_text() == 'pdid':
        tx.set_text('user share')
    if tx.get_text() == 'ndids':
        tx.set_text('users')
    if tx.get_text() == 'ncontacts_2':
        tx.set_text('in stadiums')
    if tx.get_text() == 'fpingsperdid':
        tx.set_text('ping rate')

plt.savefig(f'plots/fig2_{cdef_alt}/04_scaling_cities.jpg', bbox_inches='tight', dpi=300)
plt.savefig(f'plots/fig2_{cdef_alt}/04_scaling_cities.pdf', bbox_inches='tight')
plt.show()

In [ ]:
expo_city_p, expo_city_q = model.params.ndids, model.params.fpingsperdid# for tl7
#expo_city = 1.0566# for tl5
#expo_city = 1.0694# for tl8 10min
#expo_city = 1.1038# for tl8 60min
print(f'alpha_p: {expo_city_p}, alpha_q: {expo_city_q}')

### ... for stadiums

In [ ]:
conpop_cmp = data\
                .merge(panel_data[['city','pdid','ndids','population']], on='city')\
                .merge(panelstad_data2, on=['day','area_id'], how='left', suffixes=('','_stad'))\
                .merge(stadium_data[['area_id','capacity']], on='area_id')\
                .merge(data_pingfreq[['city','fpingsperdid']], on='city')\
                .drop_duplicates()
conpop_cmp['pdid_stad'] = conpop_cmp.ndids_stad / conpop_cmp.capacity
#conpop_cmp['ncontacts_2_normed'] = conpop_cmp.ncontacts_2 / conpop_cmp.ndids_stad**1.4701# / conpop_cmp.pdid_stad
#match_rank.pdid_stad[match_rank.ncontacts_2>=10].mean()
conpop_cmp = conpop_cmp.merge(match_data[['day','city','is_football']])

In [ ]:
cmap = sns.color_palette('flare', as_cmap=True)#sns.cubehelix_palette(rot=-.2, as_cmap=True)
g = sns.relplot(
    data=conpop_cmp,
    x="capacity", y="ncontacts_2",# x="ndids_stad",
    hue="pdid_stad", size="fpingsperdid",#"capacity",#"ncontacts_2_normed",
    palette=cmap, sizes=(10, 50),#200),
)
g.set(xscale="log", yscale="log")
g.ax.xaxis.grid(True, "minor", linewidth=.25)
g.ax.yaxis.grid(True, "minor", linewidth=.25)
g.despine(left=True, bottom=True)
g.ax.set_xlabel('stadium capacity')#'users in stadium')
g.ax.set_ylabel('detected contacts in stadium')
g.ax.grid(which='major', linestyle='-', linewidth='0.5', color='black', alpha=.25)
g.ax.grid(which='minor', linestyle='-', linewidth='0.5', color='grey', alpha=.25)
#g.ax.set_xlim([1./1.2*conpop_cmp[conpop_cmp.ncontacts_2>0.].ndids_stad.min(),1.2*conpop_cmp.ndids_stad.max()])# x=ndids_stad
g.ax.set_xlim([1./1.2*conpop_cmp[conpop_cmp.ncontacts_2>0.].capacity.min(),1.2*conpop_cmp.capacity.max()])# x=capacity
#g.ax.set_ylim([1e0,7e2])

##lg = g._legend
#for tx in lg.texts:
#    if tx.get_text() == 'pdid_stad':
#        tx.set_text('user share')
##lg.set_title('user share')

lg = g._legend
for tx in lg.texts:
    if tx.get_text() == 'pdid_stad':
        tx.set_text('user share')
    if tx.get_text() == 'ncontacts_2':
        tx.set_text('in stadiums')
    if tx.get_text() == 'fpingsperdid':
        tx.set_text('ping rate')

plt.savefig(f'plots/fig2_{cdef_alt}/04_stadbias_npop.jpg', bbox_inches='tight', dpi=300)
plt.savefig(f'plots/fig2_{cdef_alt}/04_stadbias_npop.pdf', bbox_inches='tight')
plt.show()

In [ ]:
# Define the response variable y and the predictor variables X
ncthr = 6
y = conpop_cmp[(conpop_cmp.ncontacts_2 >= ncthr) & (conpop_cmp.ndids_stad > 0)]['ncontacts_2']
X = conpop_cmp[(conpop_cmp.ncontacts_2 >= ncthr) & (conpop_cmp.ndids_stad > 0)][['ndids_stad','fpingsperdid']]#[['pdid', 'capacity']][['ndids_stad']]
#X['pdid'] = X.pdid / pdid_avg
y, X = np.log10(y), np.log10(X)
 
# Add a constant term to the predictors (for the intercept)
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X).fit()
 
# Print the model summary
print(model.summary())

In [ ]:
plt.scatter(conpop_cmp.ndids_stad, conpop_cmp.ncontacts_2)
#plt.xscale('symlog', linthresh=1)
#plt.yscale('symlog', linthresh=1)

In [ ]:
# Predict values using the model
predictions = model.predict(X)
conpop_cmp['ncontacts_2_model'] = 10.**predictions
 
# Plot true values vs predicted values
plt.scatter(y, predictions)
plt.xlabel('True Values')
plt.ylabel('Predicted Values')
plt.title('True vs Predicted Values')
plt.plot([min(y), max(y)], [min(y), max(y)], color='red', linestyle='--')  # Line of perfect prediction
plt.show()

In [ ]:
cmap = sns.color_palette('flare', as_cmap=True)#sns.cubehelix_palette(rot=-.2, as_cmap=True)
g = sns.relplot(
    data=conpop_cmp,
    x="ncontacts_2_model", y="ncontacts_2",
    hue="pdid_stad", size="fpingsperdid",
    palette=cmap, sizes=(10, 50),#200),
    lw=0,# size=10,
)
#g.ax.plot(10.**(X.ndids_stad), 10.**(predictions), c='k', lw=3)
g.ax.plot([1e0,1e5], [1e0,1e5], c='k', lw=3)
g.set(xscale="log", yscale="log")
g.ax.xaxis.grid(True, "minor", linewidth=.25)
g.ax.yaxis.grid(True, "minor", linewidth=.25)
g.despine(left=True, bottom=True)
g.ax.set_xlabel('predicted contacts in stadium')#'users in stadium')
g.ax.set_ylabel('detected contacts in stadium')
#g.ax.set_xlim([2.01e0,1.2e2])
g.ax.grid(which='major', linestyle='-', linewidth='0.5', color='black', alpha=.25)
g.ax.grid(which='minor', linestyle='-', linewidth='0.5', color='grey', alpha=.25)
#g.ax.set_xlim([1./1.2*conpop_cmp[conpop_cmp.ncontacts_2>0.].ndids_stad.min(),1.2*conpop_cmp.ndids_stad.max()])
#g.ax.set_ylim([1e0,7e2])
xu, xo = conpop_cmp.ncontacts_2_model.min() / 1.2, conpop_cmp.ncontacts_2_model.max() * 1.2
yu, yo = max(ncthr,conpop_cmp.ncontacts_2.min()) / 1.2, conpop_cmp.ncontacts_2.max() * 1.2
g.ax.set_xlim([xu, xo])
g.ax.set_ylim([yu, yo])
#g.ax.set_xticks([3e2,1e3,3e3])
#g.ax.set_xticklabels([r'$3\times 10^2$', r'$10^3$', r'$3\times 10^3$'], fontname="Arial")
#g.ax.set_yticks([3e2,1e3,3e3])
#g.ax.set_yticklabels([r'$3\times 10^2$', r'$10^3$', r'$3\times 10^3$'], fontname="Arial")

##lg = g._legend
#lg.remove()
##for tx in lg.texts:
#    if tx.get_text() == 'pdid_stad':
#        tx.set_text('user share')
##    if tx.get_text() == '10':
##        tx.set_text('')
##lg.set_title('user share')

# Retrieve the legend handles and labels
##handles, labels = lg.legend_handles, lg.texts
#print([l.get_text() for l in labels])
# Define the label you want to remove
##label_to_remove = ''
# Filter out the handle and label to remove
##filtered_handles_labels = [(h, l) for h, l in zip(handles, labels) if l.get_text() != label_to_remove]
#print(filtered_handles_labels)
# Unpack the filtered handles and labels
##filtered_handles, filtered_labels = zip(*filtered_handles_labels)
# Update the legend with the filtered handles and labels
#g.ax.legend(filtered_handles, [label.get_text() for label in filtered_labels])

lg = g._legend
for tx in lg.texts:
    if tx.get_text() == 'pdid_stad':
        tx.set_text('user share')
    if tx.get_text() == 'ndids_stad':
        tx.set_text('users')
    if tx.get_text() == 'ncontacts_2':
        tx.set_text('in stadiums')
    if tx.get_text() == 'fpingsperdid':
        tx.set_text('ping rate')

plt.savefig(f'plots/fig2_{cdef_alt}/04_scaling_stadiums.jpg', bbox_inches='tight', dpi=300)
plt.savefig(f'plots/fig2_{cdef_alt}/04_scaling_stadiums.pdf', bbox_inches='tight')
plt.show()

In [ ]:
expo_stad_p, expo_stad_q = model.params.ndids_stad, model.params.fpingsperdid# for tl7
#expo_stad = 1.7619# for tl5
#expo_stad = 1.1716# for tl8 10min
#expo_stad = 1.5373# for tl8 60min
print(f'alpha_p: {expo_stad_p}, alpha_q: {expo_stad_q}')

## sketch of extreme cases

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define lattice size
rows, cols = 5, 5
spacing = 1  # Distance between points

# Generate grid points
x, y = np.meshgrid(np.arange(cols) * spacing, np.arange(rows) * spacing)
points = np.column_stack([x.ravel(), y.ravel()])

# Plot points
fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(points[:, 0], points[:, 1], color='black', zorder=2, s=400)

# Connect neighboring points with lines
for i in range(rows):
    for j in range(cols):
        idx = i * cols + j
        if j < cols - 1:  # Connect to the right neighbor
            ax.plot([points[idx, 0], points[idx + 1, 0]], 
                    [points[idx, 1], points[idx + 1, 1]], 
                    color='black', zorder=1)
        if i < rows - 1:  # Connect to the bottom neighbor
            ax.plot([points[idx, 0], points[idx + cols, 0]], 
                    [points[idx, 1], points[idx + cols, 1]], 
                    color='black', zorder=1)

# Remove axes
ax.set_xticks([])
ax.set_yticks([])
ax.set_frame_on(False)

plt.savefig(f'plots/fig2_{cdef_alt}/04_lattice_expo1.jpg', bbox_inches='tight', dpi=300)
plt.savefig(f'plots/fig2_{cdef_alt}/04_lattice_expo1.pdf', bbox_inches='tight')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define lattice size
rows, cols = 5, 5
spacing = 1  # Distance between points

# Generate grid points
x, y = np.meshgrid(np.arange(cols) * spacing, np.arange(rows) * spacing)
points = np.column_stack([x.ravel(), y.ravel()])

# Plot points
fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(points[:, 0], points[:, 1], color='black', zorder=2, s=400)

# Fully connect all points
for i in range(len(points)):
    for j in range(i + 1, len(points)):
        ax.plot([points[i, 0], points[j, 0]], 
                [points[i, 1], points[j, 1]], 
                color='black', alpha=0.3, zorder=1)  # Use alpha for clarity

# Remove axes
ax.set_xticks([])
ax.set_yticks([])
ax.set_frame_on(False)

plt.savefig(f'plots/fig2_{cdef_alt}/04_lattice_expo2.jpg', bbox_inches='tight', dpi=300)
plt.savefig(f'plots/fig2_{cdef_alt}/04_lattice_expo2.pdf', bbox_inches='tight')
plt.show()